In [72]:
%matplotlib inline

import warnings
from collections import deque
warnings.filterwarnings('ignore')

import random
import pandas as pd
# from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
import json
import graphistry
sns.set_style('whitegrid')

In [81]:
%autosave 60

Autosaving every 60 seconds


In [73]:
file_path = "subset_res.edgelist"
df = pd.read_csv(file_path, header=None, decimal=",", delimiter=" ", names=['ID', 'neighbourID', 'edge_weight'])[1:]
print(len(df))
df.sample(3)

219139


,ID,neighbourID,edge_weight
13446,-83828358,-133797584,0.00212341
32992,-68781271,-44307118,0.00309837
42745,-21526325,-39848540,0.024979499999999998


In [74]:
ids_list = list(set(list(df["ID"].unique())+list(df["neighbourID"].unique())))
len(ids_list)

4939

In [7]:
df_data = pd.read_csv('groups_info', sep =';')

In [75]:
df_data = pd.read_csv('groups_info', sep=";",  dtype={"Name":str, "ScreenName":str, "Type":str, "Url":str, "ID":str, "MembersCount":int, "neighbourID":str})
df_data

,Name,ScreenName,Type,Url,ID,MembersCount,neighbourID
0,Никогда не сдавайся!,never_g1ve_up,2,https://vk.com/club53503880,-53503880,317713,-53503880
1,Записки Неизвестного и ОбщежитиЕ | 16 ноября,nzapiski.obshezhitie,3,https://vk.com/club60064207,-60064207,96,-60064207
2,Новоладожский судоремонтный завод,novladnlsz,1,https://vk.com/club153385562,-153385562,203,-153385562
3,"Молодёжный центр ""Зеркало"" | Красноярск",mczerkalo,2,https://vk.com/club34820906,-34820906,4145,-34820906
4,АВТОМОБИЛЬНЫЙ ТУР ПО ТОПОВЫМ МЕСТАМ СИЛЫ КРЫМА,club93008936,3,https://vk.com/club93008936,-93008936,99,-93008936
5,Бесплатная Казань,club47699809,2,https://vk.com/club47699809,-47699809,45170,-47699809
6,СВОБОДНЫЕ НОВОСТИ: ЦЕНТРАЛЬНЫЙ ОКРУГ,freedomnewscentr,1,https://vk.com/club33942198,-33942198,1280,-33942198
7,SF Education | Карьера. Финансы и Консалтинг,sfeducation,2,https://vk.com/club32342123,-32342123,73527,-32342123
8,Встречи в кругу с Татьяной Дар,club44889054,1,https://vk.com/club44889054,-44889054,170,-44889054
9,Нищева Н. В.,doshped,2,https://vk.com/club82373093,-82373093,29951,-82373093


In [10]:
result = df.copy()
result = pd.merge(result,
                 df_data[['ID', 'Name']],
                 on='ID')
result.rename(columns={"Name": "ID_name"}, inplace=True)

result = pd.merge(result,
                 df_data[['neighbourID', 'Name']],
                 on='neighbourID')
result.rename(columns={"Name": "neighbourID_name"}, inplace=True)
result.head()
len(result)
result

,ID,neighbourID,edge_weight,ID_name,neighbourID_name
0,-59068817,-110750331,0.0021646,Пироговая 29,Важные Вещи. Сток-центр распродаж детской одежды
1,-3728827,-110750331,0.00236148,Серьёзно о прививках и не только,Важные Вещи. Сток-центр распродаж детской одежды
2,-98991503,-110750331,0.00384615,"Иван - Чай ,Чага,Травы и Чай (Вологда)",Важные Вещи. Сток-центр распродаж детской одежды
3,-47683898,-110750331,0.00170736,Жесть по-Архангельски,Важные Вещи. Сток-центр распродаж детской одежды
4,-35741689,-110750331,0.0194171,ВОП — о жизни Вологды,Важные Вещи. Сток-центр распродаж детской одежды
5,-67616658,-110750331,0.00253429,САМАЯ КЛАССНАЯ ГРУППА:*)))!!!!!,Важные Вещи. Сток-центр распродаж детской одежды
6,-27311888,-110750331,0.024972900000000003,"Конный клуб БЕЛОЯР , г. Вологда",Важные Вещи. Сток-центр распродаж детской одежды
7,-104314662,-110750331,0.00230027,Объявления Барахолка Архангельск Северодвинск,Важные Вещи. Сток-центр распродаж детской одежды
8,-36848746,-110750331,0.00190779,Замена масла Шиномонтаж МарияАВТО Архангельск,Важные Вещи. Сток-центр распродаж детской одежды
9,-32012270,-110750331,0.0029558,"Куда пойти с детьми СПб, что посмотреть Питер",Важные Вещи. Сток-центр распродаж детской одежды


df.sample(3)

In [77]:
graphistry.register(key='3bd0ff5a5304a3ee27de2ca78ac7b67bbc48dc67409c255c0f70250955cf967c')
print(graphistry.__version__)

0.10.4


In [78]:
g = graphistry.edges(result).bind(source='ID_name', destination='neighbourID_name', edge_weight='edge_weight')

In [76]:
g.plot()

Uploading 7494 kB. This may take a while...


In [17]:
file_path = "subset_500_res.edgelist"
df_sample = pd.read_csv(file_path, header=None, decimal=",", delimiter=" ", names=['ID', 'neighbourID', 'edge_weight'])[1:]
print(len(df_sample))
print(df_sample.sample(3))
g = graphistry.edges(df_sample).bind(source='ID', destination='neighbourID', edge_weight='edge_weight')
g.plot()

2009
             ID neighbourID edge_weight
1446  -99545259   -43687400  0.00199203
1982   -5319899   -33755395  0.00322593
737   -33743708    -1000330  0.00267982


In [18]:
file_path = "subset_res.edgelist"
df_sample = pd.read_csv(file_path, header=None, decimal=",", delimiter=" ", names=['ID', 'neighbourID', 'edge_weight'])[1:]
print(len(df_sample))

# df_sample = pd.merge(df_sample,
#                  df_data[['ID', 'Name']],
#                  on='ID')
# df_sample.rename(columns={"Name": "ID_name"}, inplace=True)

# df_sample = pd.merge(df_sample,
#                  df_data[['neighbourID', 'Name']],
#                  on='neighbourID')
# df_sample.rename(columns={"Name": "neighbourID_name"}, inplace=True)

# print(df_sample.head())

g = graphistry.edges(result).bind(source='ID', destination='neighbourID', edge_weight='edge_weight')
g.plot()

219139
Uploading 7494 kB. This may take a while...


In [19]:
def groups2graph(graph, df):
    roster = dict()
    for index, row in df.iterrows():
        ID = row['ID']
        neighbourID = row['neighbourID']
        edge_weight = float(row['edge_weight'])
        graph.add_edge(ID, neighbourID, weight=edge_weight)
        
    return graph

In [34]:
# small_graph= nx.Graph()
# small_graph = groups2graph(graph, df[:100])

In [ ]:
graph = nx.Graph()
graph = groups2graph(graph, df)

In [ ]:
print(nx.info(graph))

In [ ]:
nx.write_gml(graph, "graph.gml")

In [ ]:
graph = nx.read_gml("graph.gml")

In [ ]:
n = graph.number_of_nodes () 
m = graph.number_of_edges () 
print ("Количество узлов:", str (n)) 
print ("Количество ребер:", str (m)) 
print ("Количество связных компонентов: ", str(nx.number_connected_components (graph)))

In [ ]:
# plt.figure (figsize = (12,8)) 
nx.draw (graph)

In [129]:
node2vec = Node2Vec(graph, dimensions=100, walk_length=15, num_walks=15, workers=10, temp_folder="/mnt")
model = node2vec.fit(window=10, min_count=1, batch_words=4)
model.wv.get_vector(list(graph.nodes)[0])



Computing transition probabilities:   0%|          | 0/4939 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 4/4939 [00:00<02:03, 39.93it/s]

Computing transition probabilities:   0%|          | 6/4939 [00:00<11:50,  6.94it/s]

Computing transition probabilities:   0%|          | 7/4939 [00:01<18:38,  4.41it/s]

Computing transition probabilities:   0%|          | 8/4939 [00:01<21:18,  3.86it/s]

Computing transition probabilities:   0%|          | 9/4939 [00:01<20:07,  4.08it/s]

Computing transition probabilities:   0%|          | 10/4939 [00:02<17:30,  4.69it/s]

Computing transition probabilities:   0%|          | 11/4939 [00:02<19:31,  4.21it/s]

Computing transition probabilities:   0%|          | 13/4939 [00:02<20:31,  4.00it/s]

Computing transition probabilities:   0%|          | 14/4939 [00:03<23:27,  3.50it/s]

Computing transition probabilities:   0%|          | 15/4939 [00:03<33:05,  2.48it/s]

Computing transition probabilities:   0%|          | 17

array([-0.70222414, -1.0055698 , -0.06078673,  0.6200772 , -0.13315734,
       -0.2272105 , -1.1842233 , -0.20610057, -0.09119889, -1.1792494 ,
        0.14438522, -1.386799  , -0.9871381 ,  0.9266985 , -0.09372827,
       -0.51982754,  1.2458024 ,  0.11264808, -1.6667987 , -1.2255327 ,
        0.10338207, -0.20218065, -0.4825452 ,  0.22090869, -0.7561282 ,
        0.7558914 , -0.2888823 , -2.7392056 , -1.0099423 , -1.5161232 ,
        0.45082697, -0.66873807,  1.2388867 , -0.1304312 ,  1.3302958 ,
       -0.80252284,  2.1343353 , -1.2267438 ,  0.46773338, -1.6285166 ,
        0.5482209 ,  0.7433285 ,  0.87714285,  0.9529118 ,  1.618586  ,
       -0.2884192 ,  0.9832043 , -1.6770521 , -0.74866706, -0.08260099,
        0.00275158,  0.03205593, -0.07699877,  1.2149591 , -2.0645514 ,
        0.29889032, -0.50076735,  0.41166583, -0.2678337 , -1.2863065 ,
       -0.07976092,  0.63398767, -1.6985599 , -0.7457646 ,  0.19076625,
        0.34578624, -1.30198   , -0.5050863 , -0.23024018, -0.75

graph: 
    Первый позиционный аргумент должен быть графом networkx. 
    Имена узлов должны быть целыми числами или строками. 
    На выходной модели они всегда будут строками.
    
dimensions:
    Размеры вложения (по умолчанию: 128)
    
walk_length: 
    Количество узлов в каждой прогулке (по умолчанию: 80)
    
num_walks: 
    Количество прогулок на узел (по умолчанию: 10)


p: Гиперпараметр, показывающий, смещен ли обход в направлении исследования внутрь (по умолчанию: 1)

q: Гиперпараметр, оказывающий, смещен ли обход в направлении исследования наружу  (по умолчанию: 1)
    
    Когда p и q оба равны 1 (по умолчанию), это, по сути, сводится к чисто случайному блужданию    

weight_key: 
    На взвешенных графиках это ключ для атрибута weight (по умолчанию: 'weight')
workers: 
    Количество рабочих для параллельного выполнения (по умолчанию: 1)

temp_folder: 
    Строковый путь, указывающий на папку для сохранения копии графика в общей памяти.
    (Предоставляется при работе с графиками, которые слишком велики, 
     чтобы поместиться в памяти во время выполнения алгоритма.)

In [ ]:
model.wv.save_word2vec_format("saved_model")

In [2]:
#Т.к. обучение занимает много времени, загрузим эмбеддинги из файла (использовалось для 65к групп)
# from gensim.models import KeyedVectors

# model = KeyedVectors.load_word2vec_format('FinalResults/small_res.emb', binary=False)
# model.wv.get_vector("110082689")

/home/alena/repos/sna_edu/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


array([ 5.06568e-01, -7.17264e-02,  1.31919e+00, -8.87469e-01,
        9.05263e-01, -3.28905e-02, -1.74266e+00, -2.75865e-01,
       -1.07608e+00,  1.21246e+00,  1.67162e-01,  1.13019e+00,
       -7.99150e-01, -1.66169e+00, -1.95406e-01, -1.36221e-01,
        6.21017e-01,  3.93591e-01, -3.81213e-01,  9.68586e-02,
       -5.91943e-02,  5.13347e-01,  4.09856e-01,  4.37756e-02,
       -4.05707e-02, -7.03835e-02, -1.57916e-01, -5.72675e-02,
        5.17322e-01, -5.09236e-01, -1.29293e-01,  5.22190e-02,
       -7.80586e-01,  3.56728e-01, -4.97181e-02, -2.43929e-01,
        2.64872e-01, -2.58917e-01,  7.62523e-02, -2.02944e-01,
       -9.93331e-02,  4.44226e-01, -1.14119e-01,  1.42753e-01,
       -4.09553e-01,  1.39739e-01, -6.28840e-02, -1.86286e-01,
        3.02092e-01,  2.81512e-01, -2.55332e-01,  5.19888e-02,
        1.20561e-02,  2.39571e-01, -2.93189e-01,  4.36481e-03,
        9.53944e-02,  1.02096e-01,  3.92286e-01,  8.31743e-02,
       -2.86053e-01, -9.88023e-02,  1.85631e-01,  1.319

In [153]:
#Список наиболее похожих вершин
node_id = 1334
sim_list = model.wv.most_similar (list(graph.nodes)[node_id])
zero_name = df_data[df_data['ID']== list(graph.nodes)[node_id]]['Name']
zero_members_count = df_data[df_data['ID']== list(graph.nodes)[node_id]]['MembersCount']
print(zero_name.values[0], zero_members_count.values[0])
for sim in sim_list:
    name = df_data[df_data['ID']== sim[0]]['Name']
    members_count = df_data[df_data['ID']== sim[0]]['MembersCount']
    print(name.values[0], sim[1], members_count.values[0])

"Новогодняя вегетарианская вечеринка" 120
ВОЗРОЖДЕНИЕ  уважения к себе. 0.9757371544837952 238
ФЕНОТИПОЛОГИЯ - по лицу узнать о человеке все 0.9545939564704895 163
Курс «Терапевт Божественного Выравнивания тела» 0.9419229626655579 165
Вега-пати ЭКО-ДАНС 23 марта! 0.9304094910621643 104
ДВА ДНЯ В ТЕМНОТЕ в йога-клубе " ТАТТВА" 0.9296668171882629 95
Курс "работа с умом" + холотропное дыхание 0.9248815774917603 149
Ремонт, полировка и замена автостекол 0.9190657138824463 933
Практический курс по Имагинативной саморегуляции 0.918916642665863 220
Практический семинар по сухому голоданию СПб 0.900908350944519 100
Красивые Новости - Подписка 0.8980250954627991 69


In [24]:
#Векторизация рёбер
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

#Получение наиболее похожих рёбер
# edges_kv =edges_embs.as_keyed_vectors () 

edges_kv = edges_embs.as_keyed_vectors()
edges_kv.most_similar(str(('-20301629', '-53313256')))


Generating edge features: 100%|██████████| 429201/429201.0 [00:02<00:00, 211101.92it/s]


[("('-5016406', '-53313256')", 0.9874381422996521),
 ("('-53313256', '-53768634')", 0.9702972769737244),
 ("('-120965895', '-53313256')", 0.968829333782196),
 ("('-133736703', '-53313256')", 0.9679967761039734),
 ("('-185076302', '-53313256')", 0.9611279964447021),
 ("('-53313256', '-53313256')", 0.9584006667137146),
 ("('-53313256', '-72906880')", 0.9534587264060974),
 ("('-109527348', '-53313256')", 0.9527057409286499),
 ("('-133736703', '-20301629')", 0.9524838328361511),
 ("('-133736703', '-5016406')", 0.9502370953559875)]

# Link Prediction

In [25]:
# Возьмём случайную выборку ребер 
edge_subset = random.sample (graph.edges (), int(0.25*graph.number_of_edges())) 

# удалим некоторые края 
G_train = graph.copy () 
G_train.remove_edges_from (edge_subset)

In [26]:
edge_subset_size = len (list (edge_subset)) 
print ("Удалено:", str (edge_subset_size)) 
print ("Осталось:", str ((m - edge_subset_size)))

Удалено: 1939
Осталось: 5820


In [44]:
prediction_jaccard = list(nx.jaccard_coefficient(G_train))
score, label = zip(*[(s, (u,v) in edge_subset) for (u,v,s) in prediction_jaccard])
prediction_jaccard[:10]

[('-87815804', '-39052865', 0.0),
 ('-87815804', '-24727990', 0.0),
 ('-87815804', '-63833364', 0.0),
 ('-87815804', '-932', 0.0),
 ('-87815804', '-13080565', 0.03125),
 ('-87815804', '-14358944', 0.09090909090909091),
 ('-87815804', '-84552952', 0.0),
 ('-87815804', '-20414651', 0.0),
 ('-87815804', '-14213228', 0.0),
 ('-87815804', '-37007424', 0.0)]

In [51]:
G_train_up = nx.Graph()

for el in prediction_jaccard:
    G_train_up.add_edge(el[0], el[1],  weight=el[2])
len(G_train_up)

926

In [55]:
G_train_df = pd.DataFrame(prediction_jaccard, columns=['ID', 'neighbourID', 'edge_weight'])
G_train_df

,ID,neighbourID,edge_weight
0,-87815804,-39052865,0.000000
1,-87815804,-24727990,0.000000
2,-87815804,-63833364,0.000000
3,-87815804,-932,0.000000
4,-87815804,-13080565,0.031250
...,...,...,...
422450,-51353113,-5115566,0.000000
422451,-51353113,-48978833,0.032258
422452,-11137800,-5115566,0.000000
422453,-11137800,-48978833,0.000000


In [57]:
g = graphistry.edges(G_train_df).bind(source='ID', destination='neighbourID', edge_weight='edge_weight')
g.plot()

In [34]:
prediction_adamic = list(nx.adamic_adar_index(G_train))
score, label = zip(*[(s, (u,v) in edge_subset) for (u,v,s) in prediction_adamic])
prediction_adamic[:10]

In [59]:
node2vec = Node2Vec(G_train, dimensions=100, walk_length=15, num_walks=15, workers=10, temp_folder="/mnt")
model = node2vec.fit(window=10, min_count=1, batch_words=4)
model.wv.get_vector(list(G_train.nodes)[0])



Computing transition probabilities:   0%|          | 0/926 [00:00<?, ?it/s]

Computing transition probabilities:   2%|▏         | 22/926 [00:00<00:04, 213.96it/s]

Computing transition probabilities:   5%|▌         | 47/926 [00:00<00:03, 221.84it/s]

Computing transition probabilities:   9%|▊         | 80/926 [00:00<00:03, 242.89it/s]

Computing transition probabilities:  12%|█▏        | 109/926 [00:00<00:03, 253.41it/s]

Computing transition probabilities:  16%|█▌        | 147/926 [00:00<00:02, 281.22it/s]

Computing transition probabilities:  21%|██        | 190/926 [00:00<00:02, 313.23it/s]

Computing transition probabilities:  24%|██▍       | 226/926 [00:00<00:02, 322.26it/s]

Computing transition probabilities:  29%|██▊       | 266/926 [00:00<00:01, 339.61it/s]

Computing transition probabilities:  35%|███▍      | 322/926 [00:00<00:01, 380.57it/s]

Computing transition probabilities:  39%|███▉      | 362/926 [00:01<00:01, 380.84it/s]

Computing transition probabilities:  47%|███

array([-1.108788  , -0.42656305, -0.86964834, -0.24332218, -0.11115439,
        0.89467454,  0.7829684 , -0.36831275,  0.19183615,  0.2689281 ,
        0.76373464,  0.4724699 , -0.3496683 , -0.10844339, -0.74054706,
       -0.00561973,  0.45586765,  0.9979788 , -0.29692176,  0.716809  ,
       -0.5158974 , -0.06095999,  0.68378544,  0.2233904 ,  0.00661482,
       -0.15272838,  0.39765432,  0.9636071 , -0.22063027, -0.54251367,
        0.6249367 , -0.21654512, -0.424277  , -0.97710246, -0.8645553 ,
        0.19164678, -0.11340064,  0.42685252, -0.30104902, -0.5535376 ,
       -0.3613467 ,  0.5461387 , -0.47066778,  0.6516365 , -0.47400045,
       -0.0126605 , -0.1709296 , -0.4478335 , -0.28634077, -0.16909875,
       -0.1929775 , -0.09864412, -0.45814142, -0.5576665 ,  0.9844584 ,
        0.332894  ,  0.07468139,  0.11823194,  0.28077465,  0.39361608,
       -0.50864506, -0.31837416,  0.21094458, -0.84050936,  0.3725797 ,
        0.93762356,  1.050397  ,  0.634298  ,  1.417073  ,  0.75

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('FinalResults/small_res.emb', binary=False) #заменить на обрезанный граф
model.wv.get_vector("110082689")

In [60]:
#мы можем получить наиболее похожее ребро, 
#которое можно использовать для прогнозирования отсутствующих ребер

from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

edges_kv = edges_embs.as_keyed_vectors()  #получаем все возможные рёбра (осторожно может быть слишком большим)
edges_kv.most_similar (str ((list(G_train.nodes)[0], list(G_train.nodes)[5])))  



Generating edge features:   0%|          | 0/429201.0 [00:00<?, ?it/s]

Generating edge features:   5%|▍         | 20467/429201.0 [00:00<00:01, 204661.90it/s]

Generating edge features:  10%|▉         | 41177/429201.0 [00:00<00:01, 205386.47it/s]

Generating edge features:  14%|█▍        | 61874/429201.0 [00:00<00:01, 205856.69it/s]

Generating edge features:  19%|█▉        | 83022/429201.0 [00:00<00:01, 207510.92it/s]

Generating edge features:  24%|██▍       | 103970/429201.0 [00:00<00:01, 208095.59it/s]

Generating edge features:  29%|██▉       | 125413/429201.0 [00:00<00:01, 209953.38it/s]

Generating edge features:  34%|███▍      | 146587/429201.0 [00:00<00:01, 210484.61it/s]

Generating edge features:  39%|███▉      | 167738/429201.0 [00:00<00:01, 210789.59it/s]

Generating edge features:  44%|████▍     | 188347/429201.0 [00:00<00:01, 209355.14it/s]

Generating edge features:  49%|████▉     | 209371/429201.0 [00:01<00:01, 209619.72it/s]

Generating edge features:  54%|█████▎   

[("('-29532853', '-76110590')", 0.944603681564331),
 ("('-112388049', '-32740480')", 0.935103178024292),
 ("('-112388049', '-36914872')", 0.9180039167404175),
 ("('-29532853', '-61969280')", 0.9155806303024292),
 ("('-29532853', '-74296475')", 0.9084420204162598),
 ("('-29532853', '-70688639')", 0.9082958698272705),
 ("('-10306586', '-29532853')", 0.9078493118286133),
 ("('-29532853', '-78678937')", 0.9047906398773193),
 ("('-112388049', '-58946399')", 0.9016870260238647),
 ("('-112388049', '-64127606')", 0.8974367380142212)]